In [1]:
import numpy as np 
import numpy.random as npr

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.decomposition import PCA
import statsmodels.api as sm
from numpy.linalg import cond

In [2]:
N=2000
D=5 # number of features
mean = np.zeros(D)
corr = 0.9

In [3]:
y_noise = 0.1
# designate the core feature
num_corefea = np.int32(D/2)
true_cause = np.arange(num_corefea).astype(int)

## generate simulated datasets with core and spurious features
The outcome model is the same in training and testing; the outcome only depends on the core feature. 

In the training set, the covariates have high correlation. In the test set, the covariates have low correlation.

In [4]:
# simulate strongly correlated features for training
train_cov = np.ones((D, D)) * corr + np.eye(D) * (1 - corr)
train_x_true = npr.multivariate_normal(mean, train_cov, size=N)
train_x_true = train_x_true * np.concatenate([-1 * np.ones(D//2), np.ones(D - D//2)])  # create both positive and negatively correlated covariates
# train_x_true = np.exp(npr.multivariate_normal(mean, train_cov, size=N)) # exponential of gaussian; no need to be gaussian

In [5]:
# simulate weakly correlated features for testing
test_cov = np.ones((D, D)) * (1 - corr) + np.eye(D) * corr
test_x_true = npr.multivariate_normal(mean, test_cov, size=N)
# test_x_true = np.exp(npr.multivariate_normal(mean, test_cov, size=N))  # exponential of gaussian; no need to be gaussian

In [6]:
# add observation noise to the x
# spurious correlation more often occurs when the signal to noise ratio is lower
x_noise = np.array(list(np.ones(num_corefea)*0.4) + list(np.ones(D-num_corefea)*0.3))

train_x = train_x_true + x_noise * npr.normal(size=[N,D])
test_x = test_x_true + x_noise * npr.normal(size=[N,D])

In [7]:
print("\ntrain X correlation\n", np.corrcoef(train_x.T))
print("\ntest X correlation\n",np.corrcoef(test_x.T))


train X correlation
 [[ 1.          0.75268472 -0.78769464 -0.79106163 -0.78975984]
 [ 0.75268472  1.         -0.7919177  -0.78895169 -0.80054701]
 [-0.78769464 -0.7919177   1.          0.8175825   0.82090032]
 [-0.79106163 -0.78895169  0.8175825   1.          0.81406298]
 [-0.78975984 -0.80054701  0.82090032  0.81406298  1.        ]]

test X correlation
 [[1.         0.05467382 0.08729385 0.1144928  0.07028164]
 [0.05467382 1.         0.10509569 0.13189927 0.06232894]
 [0.08729385 0.10509569 1.         0.08271509 0.0634688 ]
 [0.1144928  0.13189927 0.08271509 1.         0.06554416]
 [0.07028164 0.06232894 0.0634688  0.06554416 1.        ]]


In [8]:
# generate outcome
# toy model y = x + noise
truecoeff = npr.uniform(size=num_corefea) * 10
train_y = train_x_true[:,true_cause].dot(truecoeff) + y_noise * npr.normal(size=N)
test_y = test_x_true[:,true_cause].dot(truecoeff) + y_noise * npr.normal(size=N)

In [9]:
npr.uniform(size=num_corefea) * 10

array([1.70615577, 6.49670734])

# baseline naive regression on all features

In [10]:
# regularization parameter for ridge regression
alpha = 10

In [11]:
def fitcoef(cov_train, train_y, cov_test=None, test_y=None):
	# linearReg
	print("linearReg")
	reg = LinearRegression()
	reg.fit(cov_train, train_y)
	print("coef", reg.coef_, "intercept", reg.intercept_)
	print("train accuracy", reg.score(cov_train, train_y))
	if cov_test is not None:
		print("test accuracy", reg.score(cov_test, test_y))

	# # linearReg with statsmodels
	# print("linearReg with statsmodels")
	# model = sm.OLS(train_y,sm.add_constant(cov_train, prepend=False))
	# result = model.fit()
	# print(result.summary())

	# ridgeReg
	print("ridgeReg")
	reg = Ridge(alpha=alpha)
	reg.fit(cov_train, train_y)
	print("coef", reg.coef_, "intercept", reg.intercept_)
	print("train accuracy", reg.score(cov_train, train_y))
	if cov_test is not None:
		print("test accuracy", reg.score(cov_test, test_y))

all three features have coefficient different from zeuo

test accuracy degrades much from training accuracy.

In [12]:
print("\n###########################\nall features")

cov_train = np.column_stack([train_x])
cov_test = np.column_stack([test_x])

fitcoef(cov_train, train_y, cov_test, test_y)


###########################
all features
linearReg
coef [ 1.24167576  0.79333403 -0.42037391 -0.49365606 -0.50887413] intercept 0.008332298732590074
train accuracy 0.9454696265063683
test accuracy 0.5806113930532204
ridgeReg
coef [ 1.23005591  0.78951686 -0.42557064 -0.49706846 -0.51198822] intercept 0.008310760967422573
train accuracy 0.9454644693209932
test accuracy 0.5742115385591622


next consider oracle, regression only on the core feature

In [13]:
print("\n###########################\nall features")

cov_train = np.column_stack([train_x[:,true_cause]])
cov_test = np.column_stack([test_x[:,true_cause]])

fitcoef(cov_train, train_y, cov_test, test_y)


###########################
all features
linearReg
coef [1.85000989 1.42657989] intercept -0.006953404886636219
train accuracy 0.9155656206312035
test accuracy 0.8396323819690852
ridgeReg
coef [1.8420476  1.42615402] intercept -0.007200072748142924
train accuracy 0.9155589476452339
test accuracy 0.8392236133596864


## causal-rep
now try adjust for pca factor, then learn feature coefficient, construct a prediction function using the learned feature mapping, predict on the test set

In [14]:
# fit pca to high correlated training dataset
pca = PCA(n_components=1)
pca.fit(train_x)
pca.transform(train_x)

array([[-0.95888148],
       [-0.72105373],
       [-1.91685674],
       ...,
       [-0.9249342 ],
       [ 1.21119937],
       [-4.99865343]])

In [15]:
# consider features 0,1 (have to consider a subset of features; 
# alternatively one can consider features 0,2
# cannot consider all three due to colinearity issues 
# (a.k.a. violation of overlap))
print("\n###########################\ncore + spurious 1 + pca")
candidate_trainfea = train_x[:,:-1]
candidate_testfea = test_x[:,:-1]
adjust_trainC = pca.transform(train_x)
cov_train = np.column_stack([candidate_trainfea, adjust_trainC])
print("linearReg")
feareg = LinearRegression()
feareg.fit(cov_train, train_y)
print("coef", feareg.coef_, "intercept", feareg.intercept_)
print("train accuracy", feareg.score(cov_train, train_y))


###########################
core + spurious 1 + pca
linearReg
coef [ 0.72337619  0.27388803  0.09900552  0.01204839 -1.1501754 ] intercept -0.08404732181485534
train accuracy 0.9454696265063683


In [18]:
# cond(candidate_trainfea.dot(candidate_trainfea.T))

above, after adjusting for pca factor, the spurious feature 1 returns close to zero coefficient

In [30]:
# construct a prediction model using the learned 
# feature combination of "core + spurious 1"
learned_fea_train = candidate_trainfea.dot(feareg.coef_[:candidate_trainfea.shape[1]])[:,np.newaxis]
predreg = LinearRegression()
predreg.fit(learned_fea_train, train_y)
print("trainfea_coef", predreg.coef_, "intercept", predreg.intercept_)
print("trainfea accuracy", predreg.score(learned_fea_train, train_y))

trainfea_coef [2.64977992] intercept 0.04003524109076689
trainfea accuracy 0.915908607480582


In [31]:
# apply the prediction model on the test data
learned_fea_test = candidate_testfea.dot(feareg.coef_[:candidate_trainfea.shape[1]])[:,np.newaxis]
print("testfea accuracy", predreg.score(learned_fea_test, test_y))

testfea accuracy 0.8717332930238679


above, the test accuracy no longer degrades much from the training accuracy.

also note that the test accuracy is very close to the oracle accuracy.